# 토큰화 (Tokenization)

**토큰화 목적**
- 문법적 구조 이해
- 유연한 데이터 활용

In [19]:
text = "NLP is fascinating. It has many application in real-world scenarios."

In [20]:
import nltk

# 단어 토큰화
print(nltk.word_tokenize(text))

# 문장 토큰화
print(nltk.sent_tokenize(text))

# 문장별 단어 토큰화
for sent in nltk.sent_tokenize(text):
    print(nltk.word_tokenize(sent))

['NLP', 'is', 'fascinating', '.', 'It', 'has', 'many', 'application', 'in', 'real-world', 'scenarios', '.']
['NLP is fascinating.', 'It has many application in real-world scenarios.']
['NLP', 'is', 'fascinating', '.']
['It', 'has', 'many', 'application', 'in', 'real-world', 'scenarios', '.']


### Subword 토큰화
- BertTokenizer
    - 단어를 부분 단위로 쪼개어 희귀하거나 새로운 단어도 부분적으로 표현할 수 있도록 함 => 어휘 크기를 줄이고 다양한 언어 패턴 학습 가능

In [21]:
!pip install transformers

In [22]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
word = 'unhappiness'
subwords = tokenizer.tokenize(word)

##### BertTokenizer.from_pretrained('bert-base-uncased')로 로드된 설정 파일 설명
1. **`tokenizer_config.json`**  
   - 토크나이저의 기본 설정(예: `do_lower_case=True`, `model_max_length=512`)
   - 토크나이저가 어떻게 동작해야 하는지 정의

2. **`vocab.txt`**  
   - BERT의 사전(vocabulary) 파일로, 모델이 인식하는 단어 목록을 포함
   - 예: `'unhappiness'` 같은 단어가 없으면 `'un'`, `'##happiness'` 같은 서브워드로 분리

3. **`tokenizer.json`**  
   - 단어와 서브워드 토큰을 매핑하는 JSON 파일로, WordPiece 토크나이저 정보 포함

4. **`config.json`**  
   - 모델의 주요 설정값(예: hidden size, attention heads)을 포함하는 파일로, 토크나이저보다는 모델 가중치와 관련

##### 분리 과정
1. WordPiece 토크나이저는 사전에 등록된 단어 우선 사용
    - Bert의 WordPiece 토크나이저는 최대한 긴 단어를 먼저 찾으려는 방식으로 동작
    - 'happy'가 사전에 있으면 그대로 사용, 없으면 가장 긴 서브워드 조합 찾아 나눔
2. 'happy'가 사전에 포함되지 않는 경우
    - 일반적으로 'happy'는 BERT의 사전에 포함되어 있지만, 학습된 모델에 없을 수 있음
    - 'happy'가 사전에 없는 경우 사전에 있는 조각(서브워드)로 분해
3. 그래서 과정은...
    1. "unhappiness" 검색 -> 없음
    2. "un" 검색 -> 있음 -> 유지
    3. "happiness" 검색 -> 없음
    4. "ha" 검색 -> 있음
    5. "pp" 검색 -> 있음
    6. "iness" 검색 -> 있음

In [23]:
tokenizer.tokenize(text)

['nl',
 '##p',
 'is',
 'fascinating',
 '.',
 'it',
 'has',
 'many',
 'application',
 'in',
 'real',
 '-',
 'world',
 'scenarios',
 '.']

##### 문자 단위 토큰화

In [24]:
list(word)

['u', 'n', 'h', 'a', 'p', 'p', 'i', 'n', 'e', 's', 's']

### 토큰화 주의사항
1. 구두점이나 특수 문자를 단순 제외하면 안됨
2. 줄임말, 단어 내 띄어쓰기 유의

**표준 토큰화 예제 Penn Treebank Tokenization**
- 규칙 1: 하이픈으로 구성된 단어는 하나로 유지한다.
- 규칙 2: doesn't 와 같이 '가 있는 단어는 분리한다.

In [25]:
# 구두점을 제외한 단어 토큰화
import re

text = "Time flies like an arrow; fruit flies like a banana."
re.findall(r'\b\w+\b', text)

# r'' : 파이썬 raw text (이스케이핑 문자를 문자 그대로 처리)
# \b : 경계 문자 (boundary / 공백, 구두점, ...)
# \w : 글자 (word / 영문자, 숫자, _)
# \w+ : 수량자 (하나 이상)
# => 경계 문자로 감싸인 하나 이상의 글자를 전부 찾아라

['Time',
 'flies',
 'like',
 'an',
 'arrow',
 'fruit',
 'flies',
 'like',
 'a',
 'banana']

In [26]:
# WordPunctTokenizer
# 단어/구두점으로 토큰을 구분 (', - 포함 단어로 분리)
from nltk.tokenize import WordPunctTokenizer, word_tokenize

word_punct_tokenizer = WordPunctTokenizer()

text = "Don't hesitate to use well-being practices for self-care."
print(word_punct_tokenizer.tokenize(text))

print(word_tokenize(text))

['Don', "'", 't', 'hesitate', 'to', 'use', 'well', '-', 'being', 'practices', 'for', 'self', '-', 'care', '.']
['Do', "n't", 'hesitate', 'to', 'use', 'well-being', 'practices', 'for', 'self-care', '.']


In [27]:
from nltk.tokenize import TreebankWordTokenizer

treebank_word_tokenizer = TreebankWordTokenizer()
text = '''
COVID-19(전염병), Dr.Smith(의사), NASA(우주항공국) 등 특정 기관이나 명칭이 있다. \
특수 문자 또한 태그 <br>, 가격 $100.50, 2025/02/18 날짜표현에 사용될 수 있다. \
이러한 경우, $100.50을 하나의 토큰으로 유지할 필요가 있다.
'''

print(treebank_word_tokenizer.tokenize(text))

['COVID-19', '(', '전염병', ')', ',', 'Dr.Smith', '(', '의사', ')', ',', 'NASA', '(', '우주항공국', ')', '등', '특정', '기관이나', '명칭이', '있다.', '특수', '문자', '또한', '태그', '<', 'br', '>', ',', '가격', '$', '100.50', ',', '2025/02/18', '날짜표현에', '사용될', '수', '있다.', '이러한', '경우', ',', '$', '100.50을', '하나의', '토큰으로', '유지할', '필요가', '있다', '.']


### 한국어 토큰화

In [28]:
!pip install kss==5.0.0

In [29]:
# kss (Korean Sentence Splitter)
# 한국어 문장 또는 한국어/영어 혼합 문장 등에 문장단위 토큰 생성 지원
import kss

text = "배경은 1920년대의 경성부이다. 주인공이자 인력거꾼 김 첨지의 아내는 병에 걸린 지 1달 가량이 지나 있었다. 아내는 단 한 번도 약을 먹어본 적이 없는데, 그 이유는 '병이란 놈에게 약을 주어 보내면 재미를 붙여서 자꾸 온다.'는 김 첨지의 신조 때문. 멍청이, 꼰대...가 아닐수 없다. 사실 이건 핑계고, 약을 살 돈도 벌지 못하고 있었다는 이유가 더 크다."

kss.split_sentences(text)

['배경은 1920년대의 경성부이다.',
 '주인공이자 인력거꾼 김 첨지의 아내는 병에 걸린 지 1달 가량이 지나 있었다.',
 "아내는 단 한 번도 약을 먹어본 적이 없는데, 그 이유는 '병이란 놈에게 약을 주어 보내면 재미를 붙여서 자꾸 온다.'는 김 첨지의 신조 때문.",
 '멍청이, 꼰대...가 아닐수 없다.',
 '사실 이건 핑계고, 약을 살 돈도 벌지 못하고 있었다는 이유가 더 크다.']

### 품사 태깅

**pos_tag**

pos_tag는 자연어 처리(NLP)에서 단어에 품사를 태깅하는 함수로, 주로 NLTK(Natural Language Toolkit)와 같은 라이브러리에서 사용된다. <br>

- nltk pos_tag() 주요 품사 태깅<br>

1. **NN (Noun, Singular)**  
   단수 명사를 나타낸다. 하나의 사물이나 개념을 지칭한다.  
   예시: "cat", "book", "apple"

2. **NNS (Noun, Plural)**  
   복수 명사를 나타낸다. 두 개 이상의 사물이나 개념을 지칭한다.  
   예시: "cats", "books", "apples"

3. **NNP (Proper Noun, Singular)**  
   단수 고유 명사를 나타낸다. 특정한 사람, 장소 또는 조직의 이름을 지칭한다.  
   예시: "Alice", "London", "NASA"

4. **NNPS (Proper Noun, Plural)**  
   복수 고유 명사를 나타낸다. 두 개 이상의 특정한 사람, 장소 또는 조직의 이름을 지칭한다.  
   예시: "Smiths", "United Nations"

5. **VB (Verb, Base Form)**  
   동사의 원형을 나타낸다. 일반적으로 현재 시제와 함께 사용된다.  
   예시: "run", "eat", "play"

6. **VBD (Verb, Past Tense)**  
   동사의 과거형을 나타낸다.  
   예시: "ran", "ate", "played"

7. **VBG (Verb, Gerund or Present Participle)**  
   동명사 또는 현재 분사를 나타낸다. 일반적으로 "-ing" 형태이다.  
   예시: "running", "eating", "playing"

8. **VBN (Verb, Past Participle)**  
   동사의 과거 분사형을 나타낸다. 주로 완료 시제와 함께 사용된다.  
   예시: "run" (as in "has run"), "eaten", "played"

9. **VBZ (Verb, 3rd Person Singular Present)**  
   3인칭 단수 현재형 동사를 나타낸다. 주어가 3인칭 단수일 때 사용된다.  
   예시: "runs", "eats", "plays"

10. **JJ (Adjective)**  
    형용사를 나타낸다. 명사를 수식하여 그 특성을 설명한다.  
    예시: "big", "blue", "happy"

11. **JJR (Adjective, Comparative)**  
    비교급 형용사를 나타낸다. 두 개의 대상을 비교할 때 사용된다.  
    예시: "bigger", "bluer", "happier"

12. **JJS (Adjective, Superlative)**  
    최상급 형용사를 나타낸다. 세 개 이상의 대상을 비교할 때 사용된다.  
    예시: "biggest", "bluest", "happiest"

13. **RB (Adverb)**  
    부사를 나타낸다. 동사, 형용사 또는 다른 부사를 수식한다.  
    예시: "quickly", "very", "well"

14. **RBR (Adverb, Comparative)**  
    비교급 부사를 나타낸다. 두 개의 대상을 비교할 때 사용된다.  
    예시: "more quickly", "better"

15. **RBS (Adverb, Superlative)**  
    최상급 부사를 나타낸다. 세 개 이상의 대상을 비교할 때 사용된다.  
    예시: "most quickly", "best"

16. **IN (Preposition or Subordinating Conjunction)**  
    전치사 또는 종속 접속사를 나타낸다. 명사와의 관계를 나타내거나 종속절을 시작한다.  
    예시: "in", "on", "because"

17. **DT (Determiner)**  
    한정사를 나타낸다. 명사의 수와 상태를 정의한다.  
    예시: "the", "a", "some"

18. **PRP (Personal Pronoun)**  
    인칭 대명사를 나타낸다. 사람, 사물 등을 대체할 때 사용된다.  
    예시: "I", "you", "he", "they"

19. **PRP$ (Possessive Pronoun)**  
    소유 대명사를 나타낸다. 소유 관계를 나타낸다.  
    예시: "my", "your", "his", "their"

In [30]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Playdata\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Playdata\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [31]:
from nltk.tag import pos_tag

text = "Time flies like an arrow."
tokens = word_tokenize(text)
pos_tag = pos_tag(tokens)
pos_tag

[('Time', 'NNP'),
 ('flies', 'NNS'),
 ('like', 'IN'),
 ('an', 'DT'),
 ('arrow', 'NN'),
 ('.', '.')]

In [32]:
!pip install spacy

**spacy 주요 품사 태깅**

| 태그 | 설명                 | 예시                 |
|------|----------------------|----------------------|
| ADJ  | 형용사               | big, nice           |
| ADP  | 전치사               | in, to, on          |
| ADV  | 부사                 | very, well          |
| AUX  | 조동사               | is, have (조동사로 사용될 때) |
| CONJ | 접속사               | and, or             |
| DET  | 한정사/관사          | the, a              |
| INTJ | 감탄사               | oh, wow             |
| NOUN | 명사                 | dog, table          |
| NUM  | 숫자                 | one, two, 3         |
| PART | 소사                 | 'to' (to fly에서), not |
| PRON | 대명사               | he, she, it         |
| PROPN| 고유명사             | John, France        |
| PUNCT| 구두점               | ., !, ?             |
| SCONJ| 종속 접속사          | because, if         |
| SYM  | 기호                 | $, %, @             |
| VERB | 동사                 | run, eat            |
| X    | 알 수 없는 품사       | 외국어 단어, 잘못된 형식 |

In [33]:
# spacy : 자연어처리 지원 라이브러리
import spacy

spacy.cli.download('en_core_web_sm')        # 직접 다운로드 (사용 전 1회는 반드시 다운로드)
spacy_nlp = spacy.load('en_core_web_sm')    # 로드

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [34]:
tokens = spacy_nlp(text)

for token in tokens:
    print(token.text, ":", token.pos_)

Time : NOUN
flies : VERB
like : ADP
an : DET
arrow : NOUN
. : PUNCT


### KoNLPy
- 한국어 자연어 처리를 위한 라이브러리
- 형태소 분석, 품사 태깅, 텍스트 전처리 등 기능 지원
- 여러 형태소 분석기 중 적합한 분석기 선택 가능

In [35]:
!pip install konlpy

In [36]:
from konlpy.tag import Okt

text = "나는 버스를 기다리고 있다. 지각은 면하겠군!"

okt = Okt()

morphs = okt.morphs(text)
morphs

['나', '는', '버스', '를', '기다리고', '있다', '.', '지각', '은', '면', '하겠군', '!']

In [37]:
# 품사 태깅
pos_tags = okt.pos(text)
pos_tags

[('나', 'Noun'),
 ('는', 'Josa'),
 ('버스', 'Noun'),
 ('를', 'Josa'),
 ('기다리고', 'Verb'),
 ('있다', 'Adjective'),
 ('.', 'Punctuation'),
 ('지각', 'Noun'),
 ('은', 'Josa'),
 ('면', 'Noun'),
 ('하겠군', 'Verb'),
 ('!', 'Punctuation')]

In [38]:
# 명사 추출
nouns = okt.nouns(text)
nouns

['나', '버스', '지각', '면']